# Apêndice VII 	– Ntbk 02. Criação do *dataset*  consolidado - *merges* e saneamentos.ipynb

Este notebook tem por objetivo criar uma única tabela consolidada, gerada a partir dos *merges/joins* dos *datasets* gerados pelo *Ntbk 01. Coleta dos dados e transformações preliminaress* (que são resultado dos ajustes e transformações dos dataset originais coletados).

Veremos que, para isso, precisaremos promover novas transfomações, agora em relação ao conteúdo dos dataset, para garantir a uniformidade das informações.

Está estruturado da seguinte maneira:
1. Ações Preliminares;
2. Comparação entre os *datasets* a serem consolidados;
3. Obtenção do *dataset* de referência para uniformização dos nomes dos municípios;
4. *Merges* dos *datasets* de interesse;
5. Criação do *dataset* consolidado ('ntbk 02 - dataset consolidado.csv').


# 1. Ações Preliminares

## - Importação das bibliotecas de interesse.

In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

pd.set_option('max_columns',200)
pd.set_option('max_rows',100)


## - Declaração de funções que serão utilizadas no notebook.

In [2]:
def descompactar(nome_arquivo_compactado):
    # lê o arquivo zipado e extrai o conteúdo
    with ZipFile (nome_arquivo_compactado, 'r') as zip:
        zip.extractall()
        zip.printdir()        
    return "Concluído"

In [3]:
## Função utilizada para remover caracteres especiais.

def remover_caracteres_especiais(nome):
    nome = nome.replace('Á','A').replace('Â','A').replace('Ã','A').replace('À','A').replace('É','E').replace('Ê','E').replace('Í','I').replace('Ó','O').replace('Ô','O').replace('Õ','O').replace('Ú','U').replace('Ü','U').replace('Ç', 'C')
    return nome

In [4]:
## Função utilizada para uniformizar os nomes dos municípios.
## A grafia à direita ser refere à atualização mais recente publicada pelo IBGE acerca dos municípios do Brasil.

def corrigir_nomes_municipios(nome):

    nome = nome.replace('AMPARO DA SERRA-MG','AMPARO DO SERRA-MG')\
    .replace('AMPARO DE SAO FRANCISCO-SE','AMPARO DO SAO FRANCISCO-SE')\
    .replace('AUGUSTO SEVERO-RN','CAMPO GRANDE-RN') \
    .replace('ASSU-RN','ACU-RN') \
    .replace('BALNEARIO DE PICARRAS-SC','BALNEARIO PICARRAS-SC').replace('BARAO DO MONTE ALTO-MG','BARAO DE MONTE ALTO-MG') \
    .replace('BELEM DE SAO FRANCISCO-PE','BELEM DO SAO FRANCISCO-PE').replace('BIRITIBA-MIRIM-SP','BIRITIBA MIRIM-SP') \
    .replace('BOA SAUDE-RN','JANUARIO CICCO-RN')\
    .replace('BOM JESUS-GO','BOM JESUS DE GOIAS-GO').replace('BRASOPOLIS-MG','BRAZOPOLIS-MG') \
    .replace('DONA EUSEBIA-MG','DONA EUZEBIA-MG').replace('ELDORADO DOS CARAJAS-PA','ELDORADO DO CARAJAS-PA') \
    .replace('EMBU-SP','EMBU DAS ARTES-SP').replace('FLORINIA-SP','FLORINEA-SP').replace('FORTALEZA DO TABOCAO-TO','TABOCAO-TO') \
    .replace('GRAO PARA-SC','GRAO-PARA-SC')\
    .replace('IGUARACI-PE','IGUARACY-PE').replace('LAGOA DO ITAENGA-PE','LAGOA DE ITAENGA-PE') \
    .replace('MOGI-MIRIM-SP','MOGI MIRIM-SP').replace('MUNHOZ DE MELLO-PR','MUNHOZ DE MELO-PR') \
    .replace('MUQUEM DE SAO FRANCISCO-BA','MUQUEM DO SAO FRANCISCO-BA') \
    .replace("OLHO D'AGUA DOS BORGES-RN","OLHO D'AGUA DO BORGES-RN")\
    .replace('PACAEMBU DAS ARTES-SP','PACAEMBU-SP').replace('PARATI-RJ','PARATY-RJ') \
    .replace('PASSA-VINTE-MG','PASSA VINTE-MG').replace("PINGO D'AGUA-MG","PINGO-D'AGUA-MG") \
    .replace('POXOREO-MT','POXOREU-MT').replace('PRESIDENTE CASTELO BRANCO-SC','PRESIDENTE CASTELLO BRANCO-SC') \
    .replace('SANTA CRUZ DO MONTE CASTELO-PR','SANTA CRUZ DE MONTE CASTELO-PR') \
    .replace('SANTA ISABEL DO PARA-PA','SANTA IZABEL DO PARA-PA').replace('SANTA TERESINHA-BA','SANTA TEREZINHA-BA') \
    .replace("SANTANA DO LIVRAMENTO-RS","SANT'ANA DO LIVRAMENTO-RS").replace('SAO DOMINGOS DE POMBAL-PB','SAO DOMINGOS-PB') \
    .replace('SAO THOME DAS LETRAS-MG','SAO TOME DAS LETRAS-MG')\
    .replace('SAO VALERIO DA NATIVIDADE-TO','SAO VALERIO-TO').replace('TRAJANO DE MORAIS-RJ','TRAJANO DE MORAES-RJ')
      

    return nome

In [5]:
## Função para identificar se há valores fantantes, ou NaN em um dataframe

def identificar_faltantes(df):

    
    #Mostra o tamanho do meu dataset
    print("Shape do dataframe:",df.shape)
    
    #mostra em que coluna estão esses elementos faltantes
    colunas_faltantes = df.columns[df.isna().any()].tolist()
    print("Colunas que contêm valores faltantes:",colunas_faltantes)
    
    #traz o número de linhas que restariam se eu tirasse os que seriam excluídas pelo dropna
    print("Número de linhas com valores faltantes:",df.shape[0]-df.dropna().shape[0],"\n")

    
    return colunas_faltantes

# 2. Comparação entre os *datasets* a serem consolidados.

### - Leitura dos arquivo '.csv' gerados pelo *Ntbk 01. Coleta dos dados.

In [6]:
arrecadacao_2018=pd.read_csv('ntbk 01_01 - rfb arrecadacao 2018.csv')
IBGE_Empresas=pd.read_csv('ntbk 01_02 - ibge empresas 2018.csv')
IBGE_PIB=pd.read_csv('ntbk 01_03 - ibge pib 2018.csv')
IBGE_Residentes=pd.read_csv('ntbk 01_04 - ibge população residente 2018.csv')
IBGE_Censo_Idade=pd.read_csv('ntbk 01_05 - ibge censo 2010 idades.csv')
IBGE_Censo_Instrucao=pd.read_csv('ntbk 01_06 - ibge censo instrucao.csv')
IBGE_Censo_Rendimento=pd.read_csv('ntbk 01_07 - ibge censo rendimentos.csv')

print ("Shapes:")
print ('-arrecadacao_2018',arrecadacao_2018.shape)
print ('-IBGE_Empresas',IBGE_Empresas.shape)
print ('-IBGE_PIB',IBGE_PIB.shape)
print ('-IBGE_Residentes',IBGE_Residentes.shape)
print ('-IBGE_Censo_Idade',IBGE_Censo_Idade.shape)
print ('-IBGE_Censo_Instrucao',IBGE_Censo_Instrucao.shape)
print ('-IBGE_Censo_Rendimento',IBGE_Censo_Rendimento.shape)

Shapes:
-arrecadacao_2018 (5576, 2)
-IBGE_Empresas (5570, 8)
-IBGE_PIB (5570, 20)
-IBGE_Residentes (5570, 2)
-IBGE_Censo_Idade (5565, 6)
-IBGE_Censo_Instrucao (5565, 7)
-IBGE_Censo_Rendimento (5565, 10)


Com a leitura dos shapes das bases de interesse, observa-se, de pronto, que há divergências entre os valores e, possivelmente, inconsistências. 

Sabe-se que atualmente, há, no Brasil, 5570 municípios:
- Mesmo número contido nas bases indicadas por IBGE_Empresas,IBGE_PIB e IBGE_Residentes;
- No entanto a base da RFB (indicada por arrecadacao_2018) apresenta 5576 municípios;
- Já as bases indicadas por IBGE_Censo_Idade,  IBGE_Censo_Instrucao, IBGE_Censo_Rendimento contêm 5565 municípios.


# 3. Obtenção do *dataset* de referência para uniformização dos nomes dos municípios

De modo a uniformizarmos nossas bases a partir de uma referência comum e, assim, sanearmos as possíveis divergências vamos importar a tabela Códigos de Municípios mais recente do site do IBGE (que, no caso, é a de 2020).



## Dataset: Códigos dos Municípios - 2020 (DTB_2020_v2.zip)
Descrição:

A Tabela de Códigos de Municípios do IBGE apresenta a lista dos municípios brasileiros associados a um código composto de 7 dígitos, sendo os dois primeiros referentes ao código da Unidade da Federação.

É atualizada sistematicamente de forma a incluir as alterações decorrentes do desdobramento de municípios e, conseqüentemente, da criação de novos municípios, mudanças de nome dos municípios, como também de processos de fusão que resultam na extinção ou modificação de nome de algum município.

A informação mais recente é a do ano de 2020.

Trata-se de uma arquivo compactado que contém nossa base de interesse: “RELATORIO_DTB_BRASIL_MUNICIPIO.xls”

- Fonte: Site do Instituto Brasileiro de Geografia e Estatística na Internet (https://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/divisao_territorial/)


- Endereço eletrônico: https://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/divisao_territorial/2020/DTB_2020_v2.zip
Data da coleta: 19/3/2021;


- Possui metadados: não.


In [7]:
descompactar('DTB_2020_v2.zip')

File Name                                             Modified             Size
RELATORIO_DTB_BRASIL_DISTRITO.ods              2021-02-18 08:41:42       538622
RELATORIO_DTB_BRASIL_DISTRITO.xls              2021-02-18 08:42:38      2993152
RELATORIO_DTB_BRASIL_MUNICIPIO.ods             2021-02-18 08:43:28       272777
RELATORIO_DTB_BRASIL_MUNICIPIO.xls             2021-02-18 08:44:14      1323520
RELATORIO_DTB_BRASIL_SUBDISTRITO.ods           2021-02-18 08:44:58        43461
RELATORIO_DTB_BRASIL_SUBDISTRITO.xls           2021-02-18 08:47:36       230400


'Concluído'

In [8]:
nome_arquivo = 'RELATORIO_DTB_BRASIL_MUNICIPIO.xls'
IBGE_municipios=pd.read_excel(nome_arquivo)
IBGE_municipios.head(2)

,UF,Nome_UF,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município
0,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
1,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis


Como se vê, IBGE_municipios contém as 5570 linhas, como esperado.

**Para comparar com os demais dataset, precisaremos criar a coluna padrão municipio-uf.** Mas IBGE_municípios não tem a sigla dos estados.Precisaremos, então, incluir em IBGE_municipios a sigla da uf (por meio de um merge com um dataframe auxiliar criado a partir de IBGE-PIB que conterá, exclusivamente, as variáveis '03_nome_uf' e '03_uf'.

Iniciamos coletando o valores únicos para nome do estado e a sigla do estado a partir do dataframe IBGE_PIB ('03_nome_uf' e '03_uf')

In [9]:
estado_sigla = IBGE_PIB[['03_nome_uf','03_uf']].copy()
estado_sigla = estado_sigla.drop_duplicates()
estado_sigla.head()

,03_nome_uf,03_uf
0,Rondônia,RO
52,Acre,AC
74,Amazonas,AM
136,Roraima,RR
151,Pará,PA


Vamos, agora, criar uma nova coluna com a sigla dos estados em IBGE_municipios, resultado do merge com o df estado_sigla acima criado.

In [10]:
IBGE_municipios = pd.merge(IBGE_municipios, estado_sigla,how="outer", left_on='Nome_UF',right_on='03_nome_uf')
print(IBGE_municipios.shape)
IBGE_municipios.head(2)

(5570, 15)


,UF,Nome_UF,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município,03_nome_uf,03_uf
0,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste,Rondônia,RO
1,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis,Rondônia,RO


Podemos, agora, criar a coluna padronizada 'municipio-uf' e, na sequência, excluir as variáveis que já não mais serão necessárias.

In [11]:

#Agora podemos  criar a coluna padronizada 'municpipio-uf'
IBGE_municipios['municipio-uf']=IBGE_municipios['Nome_Município'].str.upper()+'-'+IBGE_municipios['03_uf'].str.upper()

# Removendo caracteres especiais da coluna recém-criada.
IBGE_municipios['municipio-uf'] = IBGE_municipios['municipio-uf'].apply(remover_caracteres_especiais)

#Podemos apagar a coluna "03_nome_uf" e "03_uf" criada com o merge no dataframe IBGE_municipios
IBGE_municipios = IBGE_municipios.drop(['03_nome_uf','03_uf'],axis=1)

IBGE_municipios.head(2)


,UF,Nome_UF,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município,municipio-uf
0,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste,ALTA FLORESTA D'OESTE-RO
1,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis,ALTO ALEGRE DOS PARECIS-RO


# 4. *Merges* dos *datasets* de interesse.

Para a criação da tabela consolidada, faremos uma sequência de merges (com os saneamentos necessários).

e modo a uniformizar os resultados, privilegiaremos sempre as informações contidas em IBGE_municipios (por se tratar da base mais atualizada). Assim:
- municípios que não mais constem dessa relação serão excluídos; 
- municípios com grafias diferentes terão seus nomes ajustados a partir dessa referência; 

O primeiro merge será entre os dataframes IBGE_municipios e arrecadacao_2018. Os demais merges terão por ponto de partida os merges anteriores.

## Merge 1 - 'IBGE_municipios' e 'arrecadacao_2018'.

In [12]:
# Iniciamos com um outer join para identificar as ocorrências que não são mútuas entre as tabelas originais
print ('Shape de IBGE_municipios',IBGE_municipios.shape)
print ('Shape de arrecadacao_2018',arrecadacao_2018.shape)
Merge_01 =  pd.merge(IBGE_municipios['municipio-uf'], arrecadacao_2018['municipio-uf'],how="outer", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_01.shape
print ('Shape de  merge_01', Merge_01.shape)

## Identificando as divergências
diferencas=Merge_01[Merge_01['_merge']!='both']
print("\n", "Diferenças encontradas", diferencas.shape)
print (diferencas['_merge'].value_counts())
diferencas

Shape de IBGE_municipios (5570, 14)
Shape de arrecadacao_2018 (5576, 2)
Shape de  merge_01 (5609, 2)

 Diferenças encontradas (72, 2)
right_only    39
left_only     33
both           0
Name: _merge, dtype: int64


,municipio-uf,_merge
197,ELDORADO DO CARAJAS-PA,left_only
259,SANTA IZABEL DO PARA-PA,left_only
436,SAO VALERIO-TO,left_only
440,TABOCAO-TO,left_only
1076,ACU-RN,left_only
1132,JANUARIO CICCO-RN,left_only
1165,OLHO D'AGUA DO BORGES-RN,left_only
1419,SAO DOMINGOS-PB,left_only
1482,BELEM DO SAO FRANCISCO-PE,left_only
1540,IGUARACY-PE,left_only


Nota-se que há, a princípio, 33 municípios que aparecem somente em PIB_municipios (indicados com o 'left_only') e outros 39 municípios que aparecem apenas em arrecadacao_2018 (indicados com 'right_only').

Uma análise mais detida, porém, mostra que parte das inconsistências se dá por divergência de grafia (por exemplo: ACU-RN vs. ASSU-RN, ou GRAO-PARA-SC vs. GRAO PARA-SC) ou por uma mudança mais significativa de nomenclaruta (como TABOCAO-TO vs. FORTALEZA DO TABOCAO-TO, ou EMBU DAS ARTES-SP vs. EMBU-SP). Há ainda situação curiosa de município com alteração total de seu nome (como o município de 'AUGUSTO SEVERO-RN' que passou a se chamar 'CAMPO GRANDE-RN')

Faremos, então, esses ajustes de grafia. Como privilegiaremos o contido em IBGE_municipios, alteraremos sempre a grafia do dataframe à direita no merge.

Há uma situação interessante: oficialmente, o município de 'BOA SAUDE-RN', que passou a se chamar 'JANUARIO CICCO' em 1953 (quando elevado a município).Emenda da Câmara Municipal n.º 01, de 02-02-1991, porém, retornou seu noe para BOA SAUDE, mas como não houve até o momento homologação por Lei Estadual, o IBGE desconsidera essa alteração. Porém todas as referências são para Boa Saúde (RN) - vide o site da prefeitura local http://www.boasaude.rn.gov.br/historia-do-municipio.html. Como estamos privilegiando o definido pelo IBGE, manteremos JANUARIO CICCO.

In [13]:
# Como no merge1 o dataframe à direita é arrecadacao_2018, será nele que faremos as adequações.
arrecadacao_2018['municipio-uf'] = arrecadacao_2018['municipio-uf'].apply(corrigir_nomes_municipios)

Outra inconsistência diz respeito ao município de 'ASSIS CHATEAUBRIAND - PB' que passou a se chamar 'RIACHAO DO BACAMARTE-PB'. Ocorre que em arrecadacao_2018 há informação de valores arrecadados para essas duas ocorrências. Somaremos então os dois valores, atribuiremos o resultado a 'RIACHAO DO BACAMARTE-PB' e excluiremos 'ASSIS CHATEAUBRIAND - PB'. 

O mesmo ocorre com CAMPO DE SANTANA-PB e TACIMA-PB; e com LIVRAMENTO DO BRUMADO - BA e LIVRAMENTO DE NOSSA SENHORA -BA; e SANTAREM-PB e JOCA CLAUDINO-PB- serão mantidos os últimos e lhes atribuído a soma das arrecadacações.

In [14]:
municipios_atuais = ['RIACHAO DO BACAMARTE-PB','TACIMA-PB','LIVRAMENTO DE NOSSA SENHORA-BA','JOCA CLAUDINO-PB']
municipios_extintos = ['ASSIS CHATEAUBRIAND-PB', 'CAMPO DE SANTANA-PB','LIVRAMENTO DO BRUMADO-BA','SANTAREM-PB']
duplas=zip(municipios_atuais, municipios_extintos)

for i in duplas:
    print("\n",i)
    
    ind_1 = arrecadacao_2018.index[arrecadacao_2018['municipio-uf']==i[0]].tolist()
    arrec_munic_1 = arrecadacao_2018.at[ind_1[0], '01_arrecadacao_2018']
    print (i[0]," - ",arrec_munic_1)
    
    ind_2 = arrecadacao_2018.index[arrecadacao_2018['municipio-uf']==i[1]].tolist()
    arrec_munic_2 = arrecadacao_2018.at[ind_2[0], '01_arrecadacao_2018']
    print (i[1]," - ",arrec_munic_2)
    
    soma_arrec = arrec_munic_1.astype(float) + arrec_munic_2.astype(float)
    print ("Soma"," - ", soma_arrec)
    
    arrecadacao_2018.at[ind_1[0], '01_arrecadacao_2018'] = soma_arrec
    print ("Novo valor para",i[0]," - ",arrecadacao_2018.at[ind_1[0],'01_arrecadacao_2018'])    


 ('RIACHAO DO BACAMARTE-PB', 'ASSIS CHATEAUBRIAND-PB')
RIACHAO DO BACAMARTE-PB  -  422067.12
ASSIS CHATEAUBRIAND-PB  -  2696361.84000009
Soma  -  3118428.9600000903
Novo valor para RIACHAO DO BACAMARTE-PB  -  3118428.9600000903

 ('TACIMA-PB', 'CAMPO DE SANTANA-PB')
TACIMA-PB  -  708332.35
CAMPO DE SANTANA-PB  -  3574894.61000052
Soma  -  4283226.96000052
Novo valor para TACIMA-PB  -  4283226.96000052

 ('LIVRAMENTO DE NOSSA SENHORA-BA', 'LIVRAMENTO DO BRUMADO-BA')
LIVRAMENTO DE NOSSA SENHORA-BA  -  12645132.32
LIVRAMENTO DO BRUMADO-BA  -  12168718.809999
Soma  -  24813851.129999
Novo valor para LIVRAMENTO DE NOSSA SENHORA-BA  -  24813851.129999

 ('JOCA CLAUDINO-PB', 'SANTAREM-PB')
JOCA CLAUDINO-PB  -  244080.8
SANTAREM-PB  -  2029605.34000023
Soma  -  2273686.1400002297
Novo valor para JOCA CLAUDINO-PB  -  2273686.1400002297


Por fim, as ocorrências 'EXTERIOR-EX' e 'ZERADO--' não são de interesse e serão excluídas quando refizermos o merge.

### Refazer o merge, mas agora como o left join.

Saneadas as inconsistências acima, o Merge1 pode ser repetido, agora como LeftJoin de modo a mantermos todas as ocorrências de IBGE_municipios e apenas as ocorrências correspondentes em "arrecadacao_2018". Teremos, assim, mantidos os 5570 municípios.

In [15]:
print ('Shape de IBGE_municipios',IBGE_municipios.shape)
print ('Shape de arrecadacao_2018',arrecadacao_2018.shape)
Merge_01 =  pd.merge(IBGE_municipios['municipio-uf'], arrecadacao_2018,how="left", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_01.shape
print ('Shape de  merge_01', Merge_01.shape)

Shape de IBGE_municipios (5570, 14)
Shape de arrecadacao_2018 (5576, 2)
Shape de  merge_01 (5570, 3)


In [16]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(Merge_01)

Shape do dataframe: (5570, 3)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [17]:
#Podemos apagar a coluna '_merge' criada pelo comando de mesmo nome.
Merge_01 = Merge_01.drop('_merge', axis=1)
Merge_01.columns

Index(['municipio-uf', '01_arrecadacao_2018'], dtype='object')



## Merge 2 - 'Merge_01' e 'IBGE_Empresas'.

Como já conhecemos boa parte das divergências, já podemos iniciar os procedimentos pela regularização das grafias dos nomes de municípios.

In [18]:
# Como no merge1 o dataframe à direita é arrecadacao_2018, será nele que faremos as adequações.
IBGE_Empresas['municipio-uf'] = IBGE_Empresas['municipio-uf'].apply(corrigir_nomes_municipios)

Seguimos fazendo um merge ('outer') para identificar eventuais divergências.

In [19]:
# Iniciamos com um outer join para identificar as ocorrências que não são mútuas entre as tabelas originais
print ('Shape de Merge_01',Merge_01.shape)
print ('Shape de IBGE_Empresas',IBGE_Empresas.shape)
Merge_02 =  pd.merge(Merge_01['municipio-uf'], IBGE_Empresas['municipio-uf'],how="outer", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_02.shape
print ('Shape de  merge_02', Merge_02.shape)

## Identificando as divergências
diferencas=Merge_02[Merge_02['_merge']!='both']
print("\n", "Diferenças encontradas", diferencas.shape)
print (diferencas['_merge'].value_counts())
diferencas

Shape de Merge_01 (5570, 2)
Shape de IBGE_Empresas (5570, 8)
Shape de  merge_02 (5570, 2)

 Diferenças encontradas (0, 2)
both          0
right_only    0
left_only     0
Name: _merge, dtype: int64


,municipio-uf,_merge


Não há divergências. Podemos fazer o merge definitivo.

### Refazer o merge, mas agora como o left join.

Não havendo divergências remanescentes, o Merge pode ser repetido, agora como LeftJoin de modo a mantermos todas as ocorrências de 'Merge_01' e apenas as ocorrências correspondentes em "IBGE_Empresas". Teremos, assim, mantidos os 5570 municípios.

In [20]:
print ('Shape de Merge_01', Merge_01.shape)
print ('Shape de IBGE_Empresas',IBGE_Empresas.shape)
Merge_02 =  pd.merge(Merge_01, IBGE_Empresas,how="left", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_02.shape
print ('Shape de  merge_02', Merge_02.shape)

Shape de Merge_01 (5570, 2)
Shape de IBGE_Empresas (5570, 8)
Shape de  merge_02 (5570, 10)


In [21]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(Merge_01)

Shape do dataframe: (5570, 2)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [22]:
#Podemos apagar a coluna '_merge' criada pelo comando de mesmo nome.
Merge_02 = Merge_02.drop('_merge', axis=1)
Merge_02.columns

Index(['municipio-uf', '01_arrecadacao_2018', '02_unidades',
       '02_unidades_atuantes_%', '02_ocupados_total',
       '02_ocupados_assalariados_%', '02_ocupados_assalariados_medio',
       '02_salario_medio_mensal_em_SM', '02_salario_medio_mensal_em_Reais'],
      dtype='object')

## Merge 3 - 'Merge_02' e 'IBGE_PIB'.

Como já conhecemos boa parte das divergências, já podemos iniciar os procedimentos pela regularização das grafias dos nomes de municípios.

In [23]:
# Como no merge1 o dataframe à direita é arrecadacao_2018, será nele que faremos as adequações.
IBGE_PIB['municipio-uf'] = IBGE_PIB['municipio-uf'].apply(corrigir_nomes_municipios)

Seguimos fazendo um merge ('outer') para identificar eventuais divergências.

In [24]:
# Iniciamos com um outer join para identificar as ocorrências que não são mútuas entre as tabelas originais
print ('Shape de Merge_02',Merge_02.shape)
print ('Shape de IBGE_PIB',IBGE_PIB.shape)
Merge_03 =  pd.merge(Merge_02['municipio-uf'], IBGE_PIB['municipio-uf'],how="outer", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_02.shape
print ('Shape de  merge_03', Merge_03.shape)

## Identificando as divergências
diferencas=Merge_03[Merge_03['_merge']!='both']
print("\n", "Diferenças encontradas", diferencas.shape)
print (diferencas['_merge'].value_counts())
diferencas

Shape de Merge_02 (5570, 9)
Shape de IBGE_PIB (5570, 20)
Shape de  merge_03 (5570, 2)

 Diferenças encontradas (0, 2)
both          0
right_only    0
left_only     0
Name: _merge, dtype: int64


,municipio-uf,_merge


Não há divergências. Podemos fazer o merge definitivo.

### Refazer o merge, mas agora como o left join.

Não havendo divergências remanescentes, o Merge pode ser repetido, agora como LeftJoin de modo a mantermos todas as ocorrências de 'Merge_02' e apenas as ocorrências correspondentes em "IBGE_PIB". Teremos, assim, mantidos os 5570 municípios.

In [25]:
print ('Shape de Merge_02', Merge_02.shape)
print ('Shape de IBGE_PIB',IBGE_PIB.shape)
Merge_03 =  pd.merge(Merge_02, IBGE_PIB,how="left", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_03.shape
print ('Shape de  merge_03', Merge_03.shape)

Shape de Merge_02 (5570, 9)
Shape de IBGE_PIB (5570, 20)
Shape de  merge_03 (5570, 29)


In [26]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(Merge_02)

Shape do dataframe: (5570, 9)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [27]:
#Podemos apagar a coluna '_merge' criada pelo comando de mesmo nome.
Merge_03 = Merge_03.drop('_merge', axis=1)
Merge_03.columns

Index(['municipio-uf', '01_arrecadacao_2018', '02_unidades',
       '02_unidades_atuantes_%', '02_ocupados_total',
       '02_ocupados_assalariados_%', '02_ocupados_assalariados_medio',
       '02_salario_medio_mensal_em_SM', '02_salario_medio_mensal_em_Reais',
       '03_nome_uf', '03_uf', '03_cod_municipio', '03_grande_regiao',
       '03_mesorregiao', '03_microregiao', '03_regiao_imediata',
       '03_regiao_intermediaria', '03_vlr_adic_bruto_total',
       '03_vlr_adic_bruto_agropecuaria_%', '03_vlr_adic_bruto_industria_%',
       '03_vlr_adic_bruto_servicos_%', '03_vlr_adic_bruto_administracao_%',
       '03_impostos_sobre_produtos', '03_pib', '03_pib_per_capita',
       '03_atividade_principal_primeira', '03_atividade_principal_segunda',
       '03_atividade_principal_terceira'],
      dtype='object')


## Merge 4 - 'Merge_03' e 'IBGE_Residentes'.

Como já conhecemos boa parte das divergências, já podemos iniciar os procedimentos pela regularização das grafias dos nomes de municípios.

In [28]:
# Como no merge1 o dataframe à direita é arrecadacao_2018, será nele que faremos as adequações.
IBGE_Residentes['municipio-uf'] = IBGE_Residentes['municipio-uf'].apply(corrigir_nomes_municipios)

Seguimos fazendo um merge ('outer') para identificar eventuais divergências.

In [29]:
# Iniciamos com um outer join para identificar as ocorrências que não são mútuas entre as tabelas originais
print ('Shape de Merge_03',Merge_03.shape)
print ('Shape de IBGE_Residentes',IBGE_Residentes.shape)
Merge_04 =  pd.merge(Merge_03['municipio-uf'], IBGE_Residentes['municipio-uf'],how="outer", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_04.shape
print ('Shape de  merge_04', Merge_04.shape)

## Identificando as divergências
diferencas=Merge_04[Merge_04['_merge']!='both']
print("\n", "Diferenças encontradas", diferencas.shape)
print (diferencas['_merge'].value_counts())
diferencas

Shape de Merge_03 (5570, 28)
Shape de IBGE_Residentes (5570, 2)
Shape de  merge_04 (5570, 2)

 Diferenças encontradas (0, 2)
both          0
right_only    0
left_only     0
Name: _merge, dtype: int64


,municipio-uf,_merge


Não há divergências. Podemos fazer o merge definitivo.

### Refazer o merge, mas agora como o left join.

Não havendo divergências remanescentes, o Merge pode ser repetido, agora como LeftJoin de modo a mantermos todas as ocorrências de 'Merge_03' e apenas as ocorrências correspondentes em "IBGE_Residentes". Teremos, assim, mantidos os 5570 municípios.

In [30]:
print ('Shape de Merge_03', Merge_03.shape)
print ('Shape de IBGE_Residentes',IBGE_Residentes.shape)
Merge_04 =  pd.merge(Merge_03, IBGE_Residentes,how="left", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_04.shape
print ('Shape de merge_04', Merge_04.shape)

Shape de Merge_03 (5570, 28)
Shape de IBGE_Residentes (5570, 2)
Shape de merge_04 (5570, 30)


In [31]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(Merge_04)

Shape do dataframe: (5570, 30)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [32]:
#Podemos apagar a coluna '_merge' criada pelo comando de mesmo nome.
Merge_04 = Merge_04.drop('_merge', axis=1)
Merge_04.columns

Index(['municipio-uf', '01_arrecadacao_2018', '02_unidades',
       '02_unidades_atuantes_%', '02_ocupados_total',
       '02_ocupados_assalariados_%', '02_ocupados_assalariados_medio',
       '02_salario_medio_mensal_em_SM', '02_salario_medio_mensal_em_Reais',
       '03_nome_uf', '03_uf', '03_cod_municipio', '03_grande_regiao',
       '03_mesorregiao', '03_microregiao', '03_regiao_imediata',
       '03_regiao_intermediaria', '03_vlr_adic_bruto_total',
       '03_vlr_adic_bruto_agropecuaria_%', '03_vlr_adic_bruto_industria_%',
       '03_vlr_adic_bruto_servicos_%', '03_vlr_adic_bruto_administracao_%',
       '03_impostos_sobre_produtos', '03_pib', '03_pib_per_capita',
       '03_atividade_principal_primeira', '03_atividade_principal_segunda',
       '03_atividade_principal_terceira', '04_populacao_residente_2018'],
      dtype='object')


## Merge 5 - 'Merge_04' e 'IBGE_Censo_Idade'.

Como já conhecemos boa parte das divergências, já podemos iniciar os procedimentos pela regularização das grafias dos nomes de municípios.

In [33]:
# Como no merge1 o dataframe à direita é arrecadacao_2018, será nele que faremos as adequações.
IBGE_Censo_Idade['municipio-uf'] = IBGE_Censo_Idade['municipio-uf'].apply(corrigir_nomes_municipios)

Seguimos fazendo um merge ('outer') para identificar eventuais divergências.

In [34]:
# Iniciamos com um outer join para identificar as ocorrências que não são mútuas entre as tabelas originais
print ('Shape de Merge_04',Merge_04.shape)
print ('Shape de IBGE_Censo_Idade',IBGE_Censo_Idade.shape)
Merge_05 =  pd.merge(Merge_04['municipio-uf'], IBGE_Censo_Idade['municipio-uf'],how="outer", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_05.shape
print ('Shape de  merge_05', Merge_05.shape)

## Identificando as divergências
diferencas=Merge_05[Merge_05['_merge']!='both']
print("\n", "Diferenças encontradas", diferencas.shape)
print (diferencas['_merge'].value_counts())
diferencas

Shape de Merge_04 (5570, 29)
Shape de IBGE_Censo_Idade (5565, 6)
Shape de  merge_05 (5570, 2)

 Diferenças encontradas (5, 2)
left_only     5
both          0
right_only    0
Name: _merge, dtype: int64


,municipio-uf,_merge
224,MOJUI DOS CAMPOS-PA,left_only
4341,BALNEARIO RINCAO-SC,left_only
4505,PESCARIA BRAVA-SC,left_only
4923,PINTO BANDEIRA-RS,left_only
5160,PARAISO DAS AGUAS-MS,left_only


Vemos aqui que há 5 (cinco) ocorrências em Merge_04 que não existem em IBGE_Censo_Idade. Tratam-se de municípios instalados após a realização do Censo de 2010. 

Assim, como a intenção é avaliar o efeito de variáveis, como as do censo, na previsão da arrecadação, esses cinco municípios deverão ser excluídos, vez que não apresentarão a totalidade das informações.

Para isso faremos um inner join.

### Refazer o merge, mas agora como o inner join.

Havendo divergências remanescentes, o Merge pode ser repetido, agora como Inner Join de modo a mantermos todas as ocorrências mútuas de 'Merge_04' e  "IBGE_Censo_Idade". Passaremos a ter, então, 5565 municípios.

In [35]:
print ('Shape de Merge_04', Merge_04.shape)
print ('Shape de IBGE_Censo_Idade',IBGE_Censo_Idade.shape)
Merge_05 =  pd.merge(Merge_04, IBGE_Censo_Idade,how="inner", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_05.shape
print ('Shape de  merge_05', Merge_05.shape)

Shape de Merge_04 (5570, 29)
Shape de IBGE_Censo_Idade (5565, 6)
Shape de  merge_05 (5565, 35)


In [36]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(Merge_05)

Shape do dataframe: (5565, 35)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [37]:
#Podemos apagar a coluna '_merge' criada pelo comando de mesmo nome.
Merge_05 = Merge_05.drop('_merge', axis=1)
Merge_05.columns

Index(['municipio-uf', '01_arrecadacao_2018', '02_unidades',
       '02_unidades_atuantes_%', '02_ocupados_total',
       '02_ocupados_assalariados_%', '02_ocupados_assalariados_medio',
       '02_salario_medio_mensal_em_SM', '02_salario_medio_mensal_em_Reais',
       '03_nome_uf', '03_uf', '03_cod_municipio', '03_grande_regiao',
       '03_mesorregiao', '03_microregiao', '03_regiao_imediata',
       '03_regiao_intermediaria', '03_vlr_adic_bruto_total',
       '03_vlr_adic_bruto_agropecuaria_%', '03_vlr_adic_bruto_industria_%',
       '03_vlr_adic_bruto_servicos_%', '03_vlr_adic_bruto_administracao_%',
       '03_impostos_sobre_produtos', '03_pib', '03_pib_per_capita',
       '03_atividade_principal_primeira', '03_atividade_principal_segunda',
       '03_atividade_principal_terceira', '04_populacao_residente_2018',
       '05_total_residentes', '05_0-19_anos_%', '05_20-39_anos_%',
       '05_40-59_anos_%', '05_60+_anos_%'],
      dtype='object')


## Merge 6 - 'Merge_05' e 'IBGE_Censo_Instrucao'.

Como já conhecemos boa parte das divergências, já podemos iniciar os procedimentos pela regularização das grafias dos nomes de municípios.

In [38]:
# Como no merge1 o dataframe à direita é arrecadacao_2018, será nele que faremos as adequações.
IBGE_Censo_Instrucao['municipio-uf'] = IBGE_Censo_Instrucao['municipio-uf'].apply(corrigir_nomes_municipios)

Seguimos fazendo um merge ('outer') para identificar eventuais divergências.

In [39]:
# Iniciamos com um outer join para identificar as ocorrências que não são mútuas entre as tabelas originais
print ('Shape de Merge_05',Merge_05.shape)
print ('Shape de IBGE_Censo_Instrucao',IBGE_Censo_Instrucao.shape)
Merge_06 =  pd.merge(Merge_05['municipio-uf'], IBGE_Censo_Instrucao['municipio-uf'],how="outer", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_06.shape
print ('Shape de  merge_06', Merge_06.shape)

## Identificando as divergências
diferencas=Merge_06[Merge_06['_merge']!='both']
print("\n", "Diferenças encontradas", diferencas.shape)
print (diferencas['_merge'].value_counts())
diferencas

Shape de Merge_05 (5565, 34)
Shape de IBGE_Censo_Instrucao (5565, 7)
Shape de  merge_06 (5565, 2)

 Diferenças encontradas (0, 2)
both          0
right_only    0
left_only     0
Name: _merge, dtype: int64


,municipio-uf,_merge


### Refazer o merge, mas agora como o inner join.

Como não há divergências remanescentes, o Merge pode ser repetido, agora como Inner Join de modo a mantermos todas as ocorrências mútuas de 'Merge_05' e  "IBGE_Censo_Instrucao". Continuaremos a ter, então, 5565 municípios.

In [40]:
print ('Shape de Merge_05', Merge_05.shape)
print ('Shape de IBGE_Censo_Instrucao',IBGE_Censo_Instrucao.shape)
Merge_06 =  pd.merge(Merge_05, IBGE_Censo_Instrucao,how="inner", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_06.shape
print ('Shape de  merge_06', Merge_06.shape)

Shape de Merge_05 (5565, 34)
Shape de IBGE_Censo_Instrucao (5565, 7)
Shape de  merge_06 (5565, 41)


In [41]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(Merge_06)

Shape do dataframe: (5565, 41)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [42]:
#Podemos apagar a coluna '_merge' criada pelo comando de mesmo nome.
Merge_06 = Merge_06.drop('_merge', axis=1)
Merge_06.columns

Index(['municipio-uf', '01_arrecadacao_2018', '02_unidades',
       '02_unidades_atuantes_%', '02_ocupados_total',
       '02_ocupados_assalariados_%', '02_ocupados_assalariados_medio',
       '02_salario_medio_mensal_em_SM', '02_salario_medio_mensal_em_Reais',
       '03_nome_uf', '03_uf', '03_cod_municipio', '03_grande_regiao',
       '03_mesorregiao', '03_microregiao', '03_regiao_imediata',
       '03_regiao_intermediaria', '03_vlr_adic_bruto_total',
       '03_vlr_adic_bruto_agropecuaria_%', '03_vlr_adic_bruto_industria_%',
       '03_vlr_adic_bruto_servicos_%', '03_vlr_adic_bruto_administracao_%',
       '03_impostos_sobre_produtos', '03_pib', '03_pib_per_capita',
       '03_atividade_principal_primeira', '03_atividade_principal_segunda',
       '03_atividade_principal_terceira', '04_populacao_residente_2018',
       '05_total_residentes', '05_0-19_anos_%', '05_20-39_anos_%',
       '05_40-59_anos_%', '05_60+_anos_%', '06_total_instrução',
       '06_Sem_Instrução_e_Fundamental_In


## Merge 7 - 'Merge_06' e 'IBGE_Censo_Rendimento'.

Como já conhecemos boa parte das divergências, já podemos iniciar os procedimentos pela regularização das grafias dos nomes de municípios.

In [43]:
# Como no merge1 o dataframe à direita é arrecadacao_2018, será nele que faremos as adequações.
IBGE_Censo_Rendimento['municipio-uf'] = IBGE_Censo_Rendimento['municipio-uf'].apply(corrigir_nomes_municipios)

Seguimos fazendo um merge ('outer') para identificar eventuais divergências.

In [44]:
# Iniciamos com um outer join para identificar as ocorrências que não são mútuas entre as tabelas originais
print ('Shape de Merge_06',Merge_06.shape)
print ('Shape de IBGE_Censo_Rendimento',IBGE_Censo_Rendimento.shape)
Merge_07 =  pd.merge(Merge_06['municipio-uf'], IBGE_Censo_Rendimento['municipio-uf'],how="outer", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_07.shape
print ('Shape de  merge_07', Merge_07.shape)

## Identificando as divergências
diferencas=Merge_07[Merge_07['_merge']!='both']
print("\n", "Diferenças encontradas", diferencas.shape)
print (diferencas['_merge'].value_counts())
diferencas

Shape de Merge_06 (5565, 40)
Shape de IBGE_Censo_Rendimento (5565, 10)
Shape de  merge_07 (5565, 2)

 Diferenças encontradas (0, 2)
both          0
right_only    0
left_only     0
Name: _merge, dtype: int64


,municipio-uf,_merge


### Refazer o merge, mas agora como o inner join.

Como não há divergências remanescentes, o Merge pode ser repetido, agora como Inner Join de modo a mantermos todas as ocorrências mútuas de 'Merge_06' e  "IBGE_Censo_Rendimento". Continuaremos a ter, então, 5565 municípios.

In [45]:
print ('Shape de Merge_06', Merge_06.shape)
print ('Shape de IBGE_Censo_Rendimento',IBGE_Censo_Rendimento.shape)
Merge_07 =  pd.merge(Merge_06, IBGE_Censo_Rendimento,how="inner", on='municipio-uf',indicator=True,suffixes=('_tab1','_tab2'))
Merge_07.shape
print ('Shape de  merge_07', Merge_07.shape)

Shape de Merge_06 (5565, 40)
Shape de IBGE_Censo_Rendimento (5565, 10)
Shape de  merge_07 (5565, 50)


In [46]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(Merge_07)

Shape do dataframe: (5565, 50)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [47]:
#Podemos apagar a coluna '_merge' criada pelo comando de mesmo nome.
Merge_07 = Merge_07.drop('_merge', axis=1)
Merge_07.columns

Index(['municipio-uf', '01_arrecadacao_2018', '02_unidades',
       '02_unidades_atuantes_%', '02_ocupados_total',
       '02_ocupados_assalariados_%', '02_ocupados_assalariados_medio',
       '02_salario_medio_mensal_em_SM', '02_salario_medio_mensal_em_Reais',
       '03_nome_uf', '03_uf', '03_cod_municipio', '03_grande_regiao',
       '03_mesorregiao', '03_microregiao', '03_regiao_imediata',
       '03_regiao_intermediaria', '03_vlr_adic_bruto_total',
       '03_vlr_adic_bruto_agropecuaria_%', '03_vlr_adic_bruto_industria_%',
       '03_vlr_adic_bruto_servicos_%', '03_vlr_adic_bruto_administracao_%',
       '03_impostos_sobre_produtos', '03_pib', '03_pib_per_capita',
       '03_atividade_principal_primeira', '03_atividade_principal_segunda',
       '03_atividade_principal_terceira', '04_populacao_residente_2018',
       '05_total_residentes', '05_0-19_anos_%', '05_20-39_anos_%',
       '05_40-59_anos_%', '05_60+_anos_%', '06_total_instrução',
       '06_Sem_Instrução_e_Fundamental_In

In [48]:
print(Merge_07.shape)
Merge_07.info()

(5565, 49)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5565 entries, 0 to 5564
Data columns (total 49 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   municipio-uf                                5565 non-null   object 
 1   01_arrecadacao_2018                         5565 non-null   float64
 2   02_unidades                                 5565 non-null   int64  
 3   02_unidades_atuantes_%                      5565 non-null   float64
 4   02_ocupados_total                           5565 non-null   int64  
 5   02_ocupados_assalariados_%                  5565 non-null   float64
 6   02_ocupados_assalariados_medio              5565 non-null   float64
 7   02_salario_medio_mensal_em_SM               5565 non-null   float64
 8   02_salario_medio_mensal_em_Reais            5565 non-null   float64
 9   03_nome_uf                                  5565 non-null   object 
 10  0

## 5. Criação do *dataset* consolidado

Cria a tabela que será a base para a exploração dos dados e aplicação dos modelos de machine learning.

In [49]:
Merge_07.to_csv('ntbk 02 - dataset consolidado.csv',index=False)